# **Distribution and performance**

This notebook:

1. Shows ingestion performance when dealing with different distributions

# Query 1

Creates 1 hash table and one round robin table with same structure and popolutes with same data

In [ ]:
CREATE TABLE [dbo].[Trip_round_robin]
(
	[DateID] [int] NOT NULL,
	[MedallionID] [int] NOT NULL,
	[HackneyLicenseID] [int] NOT NULL,
	[PickupTimeID] [int] NOT NULL,
	[DropoffTimeID] [int] NOT NULL,
	[PickupGeographyID] [int] NULL,
	[DropoffGeographyID] [int] NULL,
	[PickupLatitude] [float] NULL,
	[PickupLongitude] [float] NULL,
	[PickupLatLong] [varchar](50) NULL,
	[DropoffLatitude] [float] NULL,
	[DropoffLongitude] [float] NULL,
	[DropoffLatLong] [varchar](50) NULL,
	[PassengerCount] [int] NULL,
	[TripDurationSeconds] [int] NULL,
	[TripDistanceMiles] [float] NULL,
	[PaymentType] [varchar](50) NULL,
	[FareAmount] [money] NULL,
	[SurchargeAmount] [money] NULL,
	[TaxAmount] [money] NULL,
	[TipAmount] [money] NULL,
	[TollsAmount] [money] NULL,
	[TotalAmount] [money] NULL
)
WITH
(
	DISTRIBUTION = ROUND_ROBIN--,
	--CLUSTERED COLUMNSTORE INDEX
)
GO

CREATE TABLE [dbo].[Trip_hash]
(
	[DateID] [int] NOT NULL,
	[MedallionID] [int] NOT NULL,
	[HackneyLicenseID] [int] NOT NULL,
	[PickupTimeID] [int] NOT NULL,
	[DropoffTimeID] [int] NOT NULL,
	[PickupGeographyID] [int] NULL,
	[DropoffGeographyID] [int] NULL,
	[PickupLatitude] [float] NULL,
	[PickupLongitude] [float] NULL,
	[PickupLatLong] [varchar](50) NULL,
	[DropoffLatitude] [float] NULL,
	[DropoffLongitude] [float] NULL,
	[DropoffLatLong] [varchar](50) NULL,
	[PassengerCount] [int] NULL,
	[TripDurationSeconds] [int] NULL,
	[TripDistanceMiles] [float] NULL,
	[PaymentType] [varchar](50) NULL,
	[FareAmount] [money] NULL,
	[SurchargeAmount] [money] NULL,
	[TaxAmount] [money] NULL,
	[TipAmount] [money] NULL,
	[TollsAmount] [money] NULL,
	[TotalAmount] [money] NULL
)
WITH
(
	DISTRIBUTION = hash(DateId)--,
	--CLUSTERED COLUMNSTORE INDEX
)
GO

truncate table dbo.Trip_round_robin;
insert into dbo.Trip_round_robin 
select top(10000000)* from dbo.Trip
option (label = 'INSERT rr');

truncate table dbo.Trip_hash;
insert into dbo.Trip_hash
select top(10000000)* from dbo.Trip
option (label = 'INSERT hash');

DBCC PDW_SHOWSPACEUSED('dbo.Trip_round_robin')
DBCC PDW_SHOWSPACEUSED('dbo.Trip_hash')

Query 2

Shows duration and plans for inserts

In [ ]:
-- Query 2
select 'hash', total_elapsed_time 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT hash'
    order by start_time desc 

select 'rr', total_elapsed_time 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT rr'
    order by start_time desc 

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT hash'
    order by start_time desc 
)

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT rr'
    order by start_time desc 
)

